We are getting the parts ready in this NB

In [1]:
pip install pandas numpy scikit-learn sentence-transformers faiss-cpu joblib jupyterlab

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
import joblib
import ast

In [3]:
df_games = pd.read_csv("df_pc_final.csv")

In [4]:
df_games.head(1)

,appid,name,release_date,about_the_game,short_description,header_image,windows,mac,linux,recommendations,publishers,categories,genres,tags,pct_pos_total,num_reviews_total,ps4,ps5,xbox
0,730,Counter Strike 2,2012-08-21,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",https://shared.akamai.steamstatic.com/store_it...,True,False,True,4401572,['Valve'],"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']","{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,False,False,False


In [5]:
df_games.count()

appid                88888
name                 88888
release_date         88888
about_the_game       88743
short_description    88843
header_image         88888
windows              88888
mac                  88888
linux                88888
recommendations      88888
publishers           88888
categories           88888
genres               88888
tags                 88888
pct_pos_total        88888
num_reviews_total    88888
ps4                  88888
ps5                  88888
xbox                 88888
dtype: int64

In [6]:
df_games.columns

Index(['appid', 'name', 'release_date', 'about_the_game', 'short_description',
       'header_image', 'windows', 'mac', 'linux', 'recommendations',
       'publishers', 'categories', 'genres', 'tags', 'pct_pos_total',
       'num_reviews_total', 'ps4', 'ps5', 'xbox'],
      dtype='object')

In [7]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88888 entries, 0 to 88887
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              88888 non-null  int64 
 1   name               88888 non-null  object
 2   release_date       88888 non-null  object
 3   about_the_game     88743 non-null  object
 4   short_description  88843 non-null  object
 5   header_image       88888 non-null  object
 6   windows            88888 non-null  bool  
 7   mac                88888 non-null  bool  
 8   linux              88888 non-null  bool  
 9   recommendations    88888 non-null  int64 
 10  publishers         88888 non-null  object
 11  categories         88888 non-null  object
 12  genres             88888 non-null  object
 13  tags               88888 non-null  object
 14  pct_pos_total      88888 non-null  int64 
 15  num_reviews_total  88888 non-null  int64 
 16  ps4                88888 non-null  bool 

In [8]:
df_header_image = df_games[['appid', 'header_image']].copy()

In [9]:
df_header_image.head(5)

,appid,header_image
0,730,https://shared.akamai.steamstatic.com/store_it...
1,578080,https://shared.akamai.steamstatic.com/store_it...
2,570,https://shared.akamai.steamstatic.com/store_it...
3,488824,https://shared.akamai.steamstatic.com/store_it...
4,440,https://shared.akamai.steamstatic.com/store_it...


In [10]:
df_games.drop(columns=['header_image'], inplace=True)

In [11]:
df_games['pct_pos_total'].isnull().sum()

0

In [12]:
df_games['short_description'] = df_games['short_description'].fillna('').astype(str)

In [13]:
df_games["name"].duplicated().sum()

2

In [14]:
df_games[df_games['name'].duplicated()]

,appid,name,release_date,about_the_game,short_description,windows,mac,linux,recommendations,publishers,categories,genres,tags,pct_pos_total,num_reviews_total,ps4,ps5,xbox
67814,1963980,,2022-05-04,In a universe where there are 4 realms (Bright...,In a universe where there are 4 realms (Bright...,True,False,False,0,['GDE ANIMES'],"['Single-player', 'Multi-player', 'PvP', 'Shar...","['Action', 'Adventure', 'RPG']","{'Adventure': 67, 'Action': 62, 'RPG': 58, 'Ac...",0,0,False,False,False
71099,664210,Boxed In,2017-12-22,Gamecat Update Boxed In is more than just a pu...,'Boxed In' was designed and developed for VR a...,True,False,False,0,['Red Chain Games Ltd.'],"['Single-player', 'Steam Achievements', 'Track...","['Casual', 'Indie']","{'Casual': 103, 'Indie': 97, 'Puzzle': 94, 'VR...",0,0,True,False,False


In [15]:
def search(dataframe, column_name, search_string):
    return dataframe[dataframe[column_name].str.contains(search_string, case=False, na=False)]


In [16]:

# Example usage
search(df_games, "name", "grand theft auto")

,appid,name,release_date,about_the_game,short_description,windows,mac,linux,recommendations,publishers,categories,genres,tags,pct_pos_total,num_reviews_total,ps4,ps5,xbox
110,12210,Grand Theft Auto IV,2020-03-24,"Niko Bellic, Johnny Klebitz, and Luis Lopez al...","Niko Bellic, Johnny Klebitz and Luis Lopez all...",True,False,False,149464,['Rockstar Games'],"['Single-player', 'Multi-player', 'Partial Con...","['Action', 'Adventure']","{'Open World': 2639, 'Action': 2076, 'Automobi...",82,149489,False,False,True
812,3240220,Grand Theft Auto V,2025-03-04,Welcome to Los Santos When a young street hust...,Experience entertainment blockbusters Grand Th...,True,False,False,15444,['Rockstar Games'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure', 'Racing']","{'Action': 267, 'Open World': 263, 'Crime': 24...",54,19644,True,True,True
1699,1547000,Grand Theft Auto San Andreas – The Definitive...,2023-01-19,"Experience the blockbuster classic, updated fo...",Grand Theft Auto: San Andreas: It’s the early ...,True,False,False,7526,['Rockstar Games'],"['Single-player', 'Full controller support', '...","['Action', 'Adventure']","{'Sexual Content': 130, 'Open World': 127, 'Ac...",76,7593,True,True,False
2443,1546990,Grand Theft Auto Vice City – The Definitive E...,2023-01-19,"Experience the genre-defining classic, updated...",Welcome to the 1980s. From the decade of big h...,True,False,False,4407,['Rockstar Games'],"['Single-player', 'Full controller support', '...","['Action', 'Adventure']","{'Action': 68, 'Third-Person Shooter': 63, 'Op...",79,4433,True,True,False
2818,1546970,Grand Theft Auto III – The Definitive Edition,2023-01-19,Experience the epic open world adventure that ...,It all starts in Liberty City. With revolution...,True,False,False,3564,['Rockstar Games'],"['Single-player', 'Full controller support', '...","['Action', 'Adventure']","{'Action': 62, 'Third-Person Shooter': 57, 'Op...",66,3589,True,True,True


In [17]:
df_games = df_games[~df_games['appid'].isin([1963980, 664210])]

In [18]:
df_games.dtypes

appid                 int64
name                 object
release_date         object
about_the_game       object
short_description    object
windows                bool
mac                    bool
linux                  bool
recommendations       int64
publishers           object
categories           object
genres               object
tags                 object
pct_pos_total         int64
num_reviews_total     int64
ps4                    bool
ps5                    bool
xbox                   bool
dtype: object

In [19]:
df_games["name"] = df_games["name"].str.strip()

In [20]:
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

In [21]:
df_games['tags_cleaned'] = df_games['tags'].copy()

In [22]:
def process_weighted_tags(tag_string):
    try:
        # Safely convert the string into a Python dictionary
        tag_dict = ast.literal_eval(tag_string)
        if not isinstance(tag_dict, dict):
            return '' # Return empty if the parsed object isn't a dict
        
        weighted_tags = []
        
        for tag, count in tag_dict.items():
            # Process multi-word tags into a single token (e.g., "Hero Shooter" -> "HeroShooter")
            processed_tag = tag.replace(' ', '')
            
            # Calculate repetitions using log scaling to balance weights
            repetitions = int(round(np.log1p(count)))
            
            # Add the processed tag to the list 'repetitions' times
            weighted_tags.extend([processed_tag] * repetitions)
            
        return ' '.join(weighted_tags)
    
    except (ValueError, SyntaxError):
        # Return an empty string if the string is not a valid literal
        return ''

# Apply the function to the 'tags' column and create the new 'tags_cleaned' column
df_games['tags_cleaned'] = df_games['tags'].apply(process_weighted_tags)



In [23]:

# View the result to confirm the column was updated
df_games['tags_cleaned'].head(10)

0    FPS FPS FPS FPS FPS FPS FPS FPS FPS FPS FPS Sh...
1    Survival Survival Survival Survival Survival S...
2    FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...
3    FPS FPS FPS FPS FPS FPS FPS FPS FPS Multiplaye...
4    FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...
5    OpenWorldSurvivalCraft OpenWorldSurvivalCraft ...
6    Survival Survival Survival Survival Survival S...
7    Sandbox Sandbox Sandbox Sandbox Sandbox Sandbo...
8    FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...
9    Mythology Mythology Mythology Mythology Mythol...
Name: tags_cleaned, dtype: object

In [24]:
df_games["tags"].head(10)

0    {'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...
1    {'Survival': 14838, 'Shooter': 12727, 'Battle ...
2    {'Free to Play': 59933, 'MOBA': 20158, 'Multip...
3    {'FPS': 8082, 'Multiplayer': 6139, 'Tactical':...
4    {'Free to Play': 62868, 'Hero Shooter': 61020,...
5    {'Open World Survival Craft': 16365, 'Sandbox'...
6    {'Survival': 18592, 'Crafting': 11822, 'Multip...
7    {'Sandbox': 18706, 'Moddable': 14479, 'Multipl...
8    {'Free to Play': 2170, 'Battle Royale': 1483, ...
9    {'Mythology': 9421, 'Action RPG': 7720, 'Actio...
Name: tags, dtype: object

In [25]:
df_games.head(5)

,appid,name,release_date,about_the_game,short_description,windows,mac,linux,recommendations,publishers,categories,genres,tags,pct_pos_total,num_reviews_total,ps4,ps5,xbox,tags_cleaned
0,730,Counter Strike 2,2012-08-21,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",True,False,True,4401572,['Valve'],"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']","{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,False,False,False,FPS FPS FPS FPS FPS FPS FPS FPS FPS FPS FPS Sh...
1,578080,PUBG BATTLEGROUNDS,2017-12-21,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,True,False,False,1732007,"['KRAFTON, Inc.']","['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...","{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,True,False,False,Survival Survival Survival Survival Survival S...
2,570,Dota 2,2013-07-09,"The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",True,True,True,14337,['Valve'],"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']","{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,False,False,False,FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...
3,488824,Tom Clancy's Rainbow Six Siege,2015-12-01,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",True,False,False,1160724,['Ubisoft'],"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],"{'FPS': 8082, 'Multiplayer': 6139, 'Tactical':...",84,1168404,False,False,True,FPS FPS FPS FPS FPS FPS FPS FPS FPS Multiplaye...
4,440,Team Fortress 2,2007-10-10,'The most fun you can have online' - PC Gamer ...,Nine distinct classes provide a broad range of...,True,False,True,41587,['Valve'],"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']","{'Free to Play': 62868, 'Hero Shooter': 61020,...",89,1146642,False,False,False,FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...


In [26]:
def combine_columns(row):
    # Combine the lists from 'categories' and 'genres' into a single list
    combined_list = row['categories'] + row['genres']
    # Convert the combined list into a space-separated string
    return ''.join(combined_list)

# Apply the function to each row and create a new column
df_games['combined'] = df_games.apply(combine_columns, axis=1)

In [27]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88886 entries, 0 to 88887
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   appid              88886 non-null  int64         
 1   name               88886 non-null  object        
 2   release_date       88886 non-null  datetime64[ns]
 3   about_the_game     88741 non-null  object        
 4   short_description  88886 non-null  object        
 5   windows            88886 non-null  bool          
 6   mac                88886 non-null  bool          
 7   linux              88886 non-null  bool          
 8   recommendations    88886 non-null  int64         
 9   publishers         88886 non-null  object        
 10  categories         88886 non-null  object        
 11  genres             88886 non-null  object        
 12  tags               88886 non-null  object        
 13  pct_pos_total      88886 non-null  int64         
 14  num_reviews

In [28]:
current_year = 2025
df_games['game_age'] = current_year - df_games['release_date'].dt.year

In [29]:
df_games['reviews_per_year'] = df_games['num_reviews_total'] / (df_games['game_age'] + 1)

In [30]:
df_games['reviews_per_year'] = df_games['reviews_per_year'].round().astype(int)

In [31]:
from math import sqrt

def calculate_wilson_score(pct_pos, num_reviews, z=1.96):  # z=1.96 corresponds to a 95% confidence interval
    if num_reviews == 0:
        return 0  # Handle cases with no reviews
    
    p = pct_pos / 100  # Convert percentage to proportion
    n = num_reviews
    denominator = 1 + z**2 / n
    center_adjusted_probability = p + z**2 / (2 * n)
    adjusted_standard_deviation = sqrt((p * (1 - p) + z**2 / (4 * n)) / n)
    lower_bound = (center_adjusted_probability - z * adjusted_standard_deviation) / denominator
    return lower_bound

# Apply the Wilson Score formula to create the quality_score column
df_games['quality_score'] = df_games.apply(
    lambda row: calculate_wilson_score(row['pct_pos_total'], row['num_reviews_total']), axis=1
)

In [32]:
search(df_games, "name", "grand theft auto")

,appid,name,release_date,about_the_game,short_description,windows,mac,linux,recommendations,publishers,...,pct_pos_total,num_reviews_total,ps4,ps5,xbox,tags_cleaned,combined,game_age,reviews_per_year,quality_score
110,12210,Grand Theft Auto IV,2020-03-24,"Niko Bellic, Johnny Klebitz, and Luis Lopez al...","Niko Bellic, Johnny Klebitz and Luis Lopez all...",True,False,False,149464,['Rockstar Games'],...,82,149489,False,False,True,OpenWorld OpenWorld OpenWorld OpenWorld OpenWo...,"['Single-player', 'Multi-player', 'Partial Con...",5,24915,0.818044
812,3240220,Grand Theft Auto V,2025-03-04,Welcome to Los Santos When a young street hust...,Experience entertainment blockbusters Grand Th...,True,False,False,15444,['Rockstar Games'],...,54,19644,True,True,True,Action Action Action Action Action Action Open...,"['Single-player', 'Multi-player', 'PvP', 'Onli...",0,19644,0.533023
1699,1547000,Grand Theft Auto San Andreas – The Definitive...,2023-01-19,"Experience the blockbuster classic, updated fo...",Grand Theft Auto: San Andreas: It’s the early ...,True,False,False,7526,['Rockstar Games'],...,76,7593,True,True,False,SexualContent SexualContent SexualContent Sexu...,"['Single-player', 'Full controller support', '...",2,2531,0.750264
2443,1546990,Grand Theft Auto Vice City – The Definitive E...,2023-01-19,"Experience the genre-defining classic, updated...",Welcome to the 1980s. From the decade of big h...,True,False,False,4407,['Rockstar Games'],...,79,4433,True,True,False,Action Action Action Action Third-PersonShoote...,"['Single-player', 'Full controller support', '...",2,1478,0.777761
2818,1546970,Grand Theft Auto III – The Definitive Edition,2023-01-19,Experience the epic open world adventure that ...,It all starts in Liberty City. With revolution...,True,False,False,3564,['Rockstar Games'],...,66,3589,True,True,True,Action Action Action Action Third-PersonShoote...,"['Single-player', 'Full controller support', '...",2,1196,0.644338


In [33]:
df_games['keyword_soup'] = df_games['tags_cleaned'] + ' ' + df_games['genres'] + ' ' + df_games['categories']

In [34]:
df_games.to_csv("df_games.csv", index=False)

In [35]:
df_games['combined'] = df_games['combined'].apply(lambda x: ''.join(x))

In [36]:
import re

def clean_final_soup(s):
    """
    Cleans a malformed soup string by removing list-like characters 
    and normalizing whitespace.
    """
    # Make sure the input is a string
    s = str(s)
    
    # 1. Use regex to remove all brackets, single quotes, and commas
    s = re.sub(r"[\[\]\,'\"]", '', s)  # Escape square brackets and include double quotes
    
    # 2. Normalize whitespace: replace multiple spaces with a single space
    s = ' '.join(s.split())
    
    return s

# Apply the cleaning function to the keyword_soup column
df_games['keyword_soup'] = df_games['keyword_soup'].apply(clean_final_soup)


In [37]:

# Display the cleaned result to verify
print(df_games['keyword_soup'].head())

0    FPS FPS FPS FPS FPS FPS FPS FPS FPS FPS FPS Sh...
1    Survival Survival Survival Survival Survival S...
2    FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...
3    FPS FPS FPS FPS FPS FPS FPS FPS FPS Multiplaye...
4    FreetoPlay FreetoPlay FreetoPlay FreetoPlay Fr...
Name: keyword_soup, dtype: object


In [38]:
df_games.to_csv("df_games.csv", index=False)

In [ ]:
# Create a cleaned, single-token version of the publisher name
df_games['publisher_cleaned'] = df_games['publishers'].str.replace(' ', '').str.lower()

# Add it to the keyword soup
df_games['keyword_soup'] = df_games['keyword_soup'] + ' ' + df_games['publisher_cleaned']

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the numerical pipeline
numerical_pipeline = Pipeline([
    ('scaler', StandardScaler())  # Use StandardScaler to scale numerical features
])

# List of numerical feature columns
numerical_features = ['game_age', 'reviews_per_year', 'quality_score']

# Apply the pipeline to the numerical features
scaled_numerical_data = numerical_pipeline.fit_transform(df_games[numerical_features])

# Convert the scaled data back to a DataFrame
df_scaled_numerical = pd.DataFrame(scaled_numerical_data, columns=numerical_features)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


# Define the TfidfVectorizer with the same powerful parameters
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words='english',
    ngram_range=(1, 2)
)

# Apply the vectorizer. The output 'tfidf_matrix' is a memory-efficient sparse matrix.
# THIS IS THE FINAL STEP. Do not convert this to a dense array for the full dataset.
tfidf_matrix = tfidf_vectorizer.fit_transform(df_games['keyword_soup'])

# You can inspect the shape and type to confirm it worked
print("Shape of the sparse TF-IDF matrix:", tfidf_matrix.shape)
print("Type of the matrix:", type(tfidf_matrix))

Shape of the sparse TF-IDF matrix: (88886, 1000)
Type of the matrix: <class 'scipy.sparse._csr.csr_matrix'>


In [42]:
from sklearn.compose import ColumnTransformer

# Define the ColumnTransformer
preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, ['game_age', 'reviews_per_year', 'quality_score']),  # Apply numerical pipeline
    ('keywords', TfidfVectorizer(
        max_features=1000,
        stop_words='english',
        ngram_range=(1, 2)
    ), 'keyword_soup')  # Apply TfidfVectorizer
])

In [43]:
processed_features = preprocessor.fit_transform(df_games)

# You can inspect the shape to confirm it worked
print("Shape of the processed features sparse matrix:", processed_features.shape)

Shape of the processed features sparse matrix: (88886, 1003)


In [44]:
short_desc = df_games['short_description'].astype(str).fillna('')
detailed_desc = df_games['about_the_game'].astype(str).fillna('')


In [45]:
df_games["combined_descriptions"] = (short_desc + ". " + detailed_desc)

In [46]:
pd.set_option('display.max_rows', None)  # Show all rows

In [47]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88886 entries, 0 to 88887
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   appid                  88886 non-null  int64         
 1   name                   88886 non-null  object        
 2   release_date           88886 non-null  datetime64[ns]
 3   about_the_game         88741 non-null  object        
 4   short_description      88886 non-null  object        
 5   windows                88886 non-null  bool          
 6   mac                    88886 non-null  bool          
 7   linux                  88886 non-null  bool          
 8   recommendations        88886 non-null  int64         
 9   publishers             88886 non-null  object        
 10  categories             88886 non-null  object        
 11  genres                 88886 non-null  object        
 12  tags                   88886 non-null  object        
 13  pct_po

In [50]:
df_games.to_csv("df_games.csv", index=False)

In [48]:
df_games.columns.tolist()

['appid',
 'name',
 'release_date',
 'about_the_game',
 'short_description',
 'windows',
 'mac',
 'linux',
 'recommendations',
 'publishers',
 'categories',
 'genres',
 'tags',
 'pct_pos_total',
 'num_reviews_total',
 'ps4',
 'ps5',
 'xbox',
 'tags_cleaned',
 'combined',
 'game_age',
 'reviews_per_year',
 'quality_score',
 'keyword_soup',
 'publisher_cleaned',
 'combined_descriptions']

In [49]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained sentence-transformers model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Apply the model to your new 'combined_description' column
embeddings = model.encode(
    df_games['combined_descriptions'].astype(str).tolist(),
    batch_size=128,
    show_progress_bar=True
)

# Convert the embeddings into a dense matrix
embeddings_matrix = np.array(embeddings)

# Display the shape of the embeddings matrix
print("Shape of the embeddings matrix:", embeddings_matrix.shape)

Batches:   0%|          | 0/695 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Save the embeddings matrix to a .npy file
np.save("embeddings_matrix.npy", embeddings_matrix)
print("Embeddings matrix saved successfully.")

Embeddings matrix saved successfully.


In [ ]:
# Load the embeddings matrix from the .npy file
embeddings_matrix = np.load("embeddings_matrix.npy")
print("Embeddings matrix loaded successfully.")

Embeddings matrix loaded successfully.


In [ ]:
def search(dataframe, column_name, search_string):
    """
    Searches for rows in the dataframe where the specified column contains the search string.
    
    Args:
        dataframe (pd.DataFrame): The dataframe to search.
        column_name (str): The name of the column to search in.
        search_string (str): The string to search for.
    
    Returns:
        pd.DataFrame: A filtered dataframe containing rows that match the search criteria.
    """
    return dataframe[dataframe[column_name].str.contains(search_string, case=False, na=False)]

In [ ]:
from torch import cuda
print(cuda.is_available()) 

False


In [ ]:
from scipy.sparse import hstack

# Convert the sparse matrix from ColumnTransformer to a sparse format if needed
processed_features_sparse = processed_features  # Ensure it's in sparse format

# Horizontally stack the sparse matrix and the dense matrix
hybrid_feature_vectors = hstack([processed_features_sparse, embeddings_matrix])

# Display the shape of the hybrid feature vectors matrix
print("Shape of the hybrid feature vectors matrix:", hybrid_feature_vectors.shape)

Shape of the hybrid feature vectors matrix: (88886, 1387)


In [ ]:
# Convert the sparse matrix to a dense array
dense_array = hybrid_feature_vectors.toarray()

# Ensure the array is in float32 format
vectors = dense_array.astype('float32')

# Display the shape of the final dense array
print("Shape of the dense vectors array:", vectors.shape)

Shape of the dense vectors array: (88886, 1387)


In [ ]:
# Define the number of dimensions
d = vectors.shape[1]

# Define the number of neighbors to retrieve
k = 6  # Item itself + 5 recommendations

# Display the values of d and k
print(f"Number of dimensions (d): {d}")
print(f"Number of neighbors (k): {k}")

Number of dimensions (d): 1387
Number of neighbors (k): 6


In [ ]:
import faiss
import time

# Record the start time
start_time = time.time()

# Initialize the index
index_flat = faiss.IndexFlatL2(d)  # L2 distance metric

# Add vectors to the index
index_flat.add(vectors)

# Record the end time


In [ ]:
end_time = time.time()

# Calculate the build time
build_time_flat = end_time - start_time

# Print the result
print(f"Time taken to build the index: {build_time_flat:.2f} seconds")

Time taken to build the index: 0.47 seconds


In [ ]:
import numpy as np

# Assuming 'vectors' is your (88886, 1387) array

# --- 1. Define Parameters ---
d_original = vectors.shape[1]  # This is 1387
m = 8

# --- 2. Calculate Padding ---
if d_original % m != 0:
    # Find the next multiple of m
    d_new = (d_original // m + 1) * m
    padding_size = d_new - d_original # This will be 5
    
    print(f"Original dimension {d_original} is not divisible by m={m}.")
    print(f"Padding with {padding_size} zeros to new dimension {d_new}.")

    # Create an array of zeros for padding
    padding = np.zeros((vectors.shape[0], padding_size), dtype='float32')
    
    # Add the padding to the original vectors
    vectors = np.hstack([vectors, padding])

# Now, 'vectors' has the shape (88886, 1392), and you can proceed
d = vectors.shape[1] # This is now 1392

Original dimension 1387 is not divisible by m=8.
Padding with 5 zeros to new dimension 1392.


: 

In [ ]:
import faiss
import numpy as np

# Ensure vectors is loaded and has the correct type
vectors = vectors.astype('float32')  # Convert to float32 if necessary
print("Shape of vectors:", vectors.shape)
print("Data type of vectors:", vectors.dtype)

# Define dimensions and parameters
d = vectors.shape[1]  # Number of dimensions
nlist = 1024          # Number of clusters
m = 8                 # Number of sub-quantizers
nbits = 8             # Number of bits per sub-quantizer

# Build the index structure
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)

# Train the index
print("Training the production index...")
index.train(vectors)

# Add vectors to the index
print("Adding vectors to the index...")
index.add(vectors)

# Save the index
print("Saving index to 'production_games.index'...")
faiss.write_index(index, "production_games.index")

print("Production index created successfully.")

In [ ]:
import numpy as np

# Save the hybrid vectors to an .npy file
np.save("hybrid_vectors.npy", vectors)

# Verify the file was saved
print("Hybrid vectors saved to 'hybrid_vectors.npy'")

Hybrid vectors saved to 'hybrid_vectors.npy'


In [ ]:
import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
# Make sure these file paths are correct
index = faiss.read_index("production_games.index")
df_games = pd.read_csv("df_games.csv") # Or your full cleaned dataset CSV
vectors = np.load("hybrid_vectors.npy") # Assuming you saved the vectors array

# Create a mapping from game title to its index position
title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_simple_recommendations(game_title, k=6):
    """
    Finds and prints recommendations for a single game title.
    """
    try:
        # 1. Get the index of the input game
        idx = title_to_index[game_title]
        
        # 2. Get the vector for that game
        query_vector = np.array([vectors[idx]], dtype='float32')
        
        # 3. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for '{game_title}' ---")
        
        # 4. Print the results (start from 1 to skip the game itself)
        for i in range(1, k):
            recommended_game_index = indices[0][i]
            recommended_game_name = df_games.iloc[recommended_game_index]['name']
            print(f"{i}. {recommended_game_name}")
            
    except KeyError:
        print(f"Error: Game '{game_title}' not found in the dataset.")

# --- Now, test the engine! ---
# Pick a game title from your dataset to test
test_game = "Call of Duty" 
get_simple_recommendations(test_game)

--- Recommendations for 'Call of Duty' ---
1. Red Dead Redemption 2
2. Destiny 2
3. Satisfactory
4. Monster Hunter Wilds
5. Tom Clancy's Rainbow Six Siege


In [ ]:
#engine 1

import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
index = faiss.read_index("production_games.index")
df_games = pd.read_csv("df_games.csv") # Or your full cleaned dataset CSV
vectors = np.load("hybrid_vectors.npy")

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, k=6):
    """
    Finds and prints recommendations based on an average profile of input games.
    """
    try:
        # 1. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]
        
        # 2. Calculate the average vector to create the "taste profile"
        query_vector = np.mean(input_vectors, axis=0).reshape(1, -1).astype('float32')
        
        # 3. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 4. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
             print(f"{i+1}. {rec}")

            
    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

# --- Now, test with both games! ---
test_games = ["Call of Duty Warzone", "Ghost of Tsushima DIRECTOR'S CUT"]
get_profile_recommendations(test_games)

--- Recommendations for a fan of Call of Duty Warzone, Ghost of Tsushima DIRECTOR'S CUT ---
1. FragPunk
2. Palia
3. Goose Goose Duck
4. REMNANT II
5. TEKKEN 8
6. One armed robber


In [ ]:
from sklearn.preprocessing import normalize
from scipy.sparse import hstack
import numpy as np

# --- 1. Define Your Weights ---
# You can tune these values to change the recommendations
keyword_weight = 2.0
# This weight is for the keyword features extracted by TfidfVectorizer
semantic_weight = 2.0
numerical_weight = 3.0

# --- 2. Separate and Apply Weights to Your Processed Features ---

# Get the number of keyword features from your TfidfVectorizer
num_keyword_features = len(tfidf_vectorizer.get_feature_names_out())

# Separate the keyword vectors from the numerical vectors
keyword_vectors = processed_features[:, :num_keyword_features]
numerical_vectors = processed_features[:, num_keyword_features:]

# Apply the weights
weighted_keywords = keyword_vectors * keyword_weight
weighted_numericals = numerical_vectors * numerical_weight
weighted_semantics = embeddings_matrix * semantic_weight


# --- 3. Combine the Weighted Vectors ---

# Combine all the weighted features into one matrix
# Note: We convert the dense numerical and semantic vectors to a sparse format for hstack
from scipy.sparse import csr_matrix
weighted_hybrid_vectors = hstack([
    weighted_keywords,
    csr_matrix(weighted_numericals),
    csr_matrix(weighted_semantics)
])


# --- 4. Normalize the Final Vectors (Crucial Step) ---

# Normalizing ensures that the vector lengths don't affect similarity calculations
# This makes the L2 distance used by Faiss equivalent to cosine similarity
final_vectors_normalized = normalize(weighted_hybrid_vectors, norm='l2', axis=1)


# --- 5. Convert to the Final Format for Faiss ---

# Convert back to a dense float32 array for Faiss
# This is your new and improved set of vectors
final_vectors = final_vectors_normalized.toarray().astype('float32')

print("Successfully created improved, weighted feature vectors.")
print("New shape:", final_vectors.shape)

Successfully created improved, weighted feature vectors.
New shape: (88886, 1387)


In [ ]:
np.save("improved_hybrid_vectors.npy", final_vectors)
print("Improved hybrid vectors saved successfully.")

Improved hybrid vectors saved successfully.


In [ ]:
print(df_games['name'].head())

0                  Counter Strike 2
1                PUBG BATTLEGROUNDS
2                            Dota 2
3    Tom Clancy's Rainbow Six Siege
4                   Team Fortress 2
Name: name, dtype: object


In [ ]:
df_games['name'] = df_games['name'].str.strip()

In [ ]:
# Replace 'specific_appid' with the appid you want to search for
specific_appid = 240  # Example appid

# Filter the DataFrame to find the row with the given appid
game_row = df_games[df_games['appid'] == specific_appid]

# Check if the appid exists in the dataset
if not game_row.empty:
    game_name = game_row.iloc[0]['name']
    print(f"The name of the game with appid {specific_appid} is: {game_name}")
else:
    print(f"No game found with appid {specific_appid}.")

The name of the game with appid 240 is: Counter Strike Source


In [ ]:
import re
# Replace multiple spaces with a single space and strip leading/trailing spaces
df_games['name'] = df_games['name'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [ ]:
df_games.to_csv("df_games.csv", index=False)

In [ ]:
#engine 2

import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("improved_hybrid_vectors.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, k=6):
    """
    Finds and prints recommendations based on an average profile of input games.
    """
    try:
        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return
        
        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]
        
        # 3. Calculate the average vector to create the "taste profile"
        query_vector = np.mean(input_vectors, axis=0).reshape(1, -1).astype('float32')
        
        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # New line: Increase the search accuracy
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

# --- Now, test with both games! ---
test_games = ["EA SPORTS FIFA 21", "EA SPORTS FC 24", "EA SPORTS FC 25"]
get_profile_recommendations(test_games)

Shape of vectors: (88886, 1387)
Shape of query vector: (1, 1387)
--- Recommendations for a fan of EA SPORTS FIFA 21, EA SPORTS FC 24, EA SPORTS FC 25 ---
1. FIFA 22
2. EA SPORTS FIFA 23
3. Aimlabs
4. It Takes Two


In [ ]:
#engine 3
import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("improved_hybrid_vectors.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6): # Added 'ratings' parameter
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return
        
        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]
        
        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')
        
        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # New line: Increase the search accuracy
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

# --- Now, test with both games! ---
test_games = ["Red Dead Redemption 2", "Grand Theft Auto V", "Watch Dogs Legion"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [10, 9, 5]
# Call the function with the new ratings parameters
get_profile_recommendations(test_games, test_ratings)


Shape of vectors: (88886, 1387)
Shape of query vector: (1, 1387)
--- Recommendations for a fan of Red Dead Redemption 2, Grand Theft Auto V, Watch Dogs Legion ---
1. Mafia II Classic
2. V Rising
3. Enshrouded
4. Dark and Darker
5. FOR HONOR


In [ ]:
import faiss
import numpy as np
import pandas as pd

# --- 1. Load All Production Components ---
try:
    # Load the fast, production-ready index you saved earlier
    index = faiss.read_index("production_games.index")
    df_games = pd.read_csv("df_games.csv")
    vectors = np.load("improved_hybrid_vectors.npy")  # Or your latest vector file
except FileNotFoundError as e:
    print(f"Error loading files: {e}. Please ensure all necessary files are in the same directory.")
    exit()

# --- 2. Create a Robust Lookup Map ---
df_games['name_cleaned'] = df_games['name'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)
title_to_index = pd.Series(df_games.index, index=df_games['name_cleaned'])

def get_weighted_recommendations(game_titles, ratings, k=6):
    """
    Generates personalized recommendations using the fast IndexIVFPQ.
    """
    if len(game_titles) != len(ratings):
        print("Error: The number of games and ratings must be the same.")
        return

    input_vectors = []
    valid_ratings = []
    for title, rating in zip(game_titles, ratings):
        cleaned_title = title.lower().strip()
        if cleaned_title in title_to_index:
            input_vectors.append(vectors[title_to_index[cleaned_title]])
            valid_ratings.append(rating)
        else:
            print(f"Warning: '{title}' not found. Skipping.")

    if not input_vectors:
        print("Error: None of the input games were found.")
        return

    # Calculate the weighted average vector to create the "taste profile"
    query_vector = np.average(input_vectors, axis=0, weights=valid_ratings).reshape(1, -1).astype('float32')

    # Debugging: Print shapes
    print(f"Query vector shape: {query_vector.shape}")
    print(f"Faiss index dimensionality: {index.d}")

    # Apply padding if necessary
    if query_vector.shape[1] < index.d:
        padding_size = index.d - query_vector.shape[1]
        query_vector = np.hstack([query_vector, np.zeros((query_vector.shape[0], padding_size), dtype='float32')])

    # Perform the search
    index.nprobe = 10
    distances, indices = index.search(query_vector, k)

    print(f"\n--- Recommendations based on your input ---")
    recs_found = 0
    input_titles_cleaned = {title.lower().strip() for title in game_titles}

    for i in range(k):
        rec_index = indices[0][i]
        rec_title_cleaned = df_games.iloc[rec_index]['name_cleaned']

        if rec_title_cleaned not in input_titles_cleaned:
            recs_found += 1
            print(f"{recs_found}. {df_games.iloc[rec_index]['name']}")

# --- Now, test with your three games and ratings! ---
test_games = ["EA SPORTS FIFA 21", "EA SPORTS FC 24", "EA SPORTS FC 25"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [10, 10, 10]

get_weighted_recommendations(test_games, test_ratings)

Query vector shape: (1, 1387)
Faiss index dimensionality: 1392

--- Recommendations based on your input ---
1. Lost Light
2. Battle Teams 2
3. GrandChase
4. Bless Unleashed
5. UNDECEMBER
6. A V A Global


In [ ]:
#engine 5

import faiss
import numpy as np
import pandas as pd

# --- Load All Production Components ---
try:
    index = faiss.read_index("production_games.index")
    df_games = pd.read_csv("df_games.csv")
    vectors = np.load("improved_version_2.npy")
except FileNotFoundError as e:
    print(f"Error loading files: {e}.")
    exit()

# --- Create a Robust Lookup Map ---
df_games['name_cleaned'] = df_games['name'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)
title_to_index = pd.Series(df_games.index, index=df_games['name_cleaned'])

def get_reranked_recommendations(game_titles, ratings, num_recs=5):
    """
    Generates personalized recommendations by finding a large pool of similar
    candidates and re-ranking them by their quality score.
    """
    if len(game_titles) != len(ratings):
        print("Error: The number of games and ratings must be the same.")
        return

    # --- Create Weighted User Profile ---
    input_vectors = []
    valid_ratings = []
    for title, rating in zip(game_titles, ratings):
        cleaned_title = title.lower().strip()
        if cleaned_title in title_to_index:
            input_vectors.append(vectors[title_to_index[cleaned_title]])
            valid_ratings.append(rating)
        else:
            print(f"Warning: '{title}' not found. Skipping.")

    if not input_vectors:
        print("Error: None of the input games were found.")
        return

    query_vector = np.average(input_vectors, axis=0, weights=valid_ratings).reshape(1, -1).astype('float32')

    # Debugging: Print shapes
    print(f"Query vector shape: {query_vector.shape}")
    print(f"Faiss index dimensionality: {index.d}")

    # Apply padding if necessary
    if query_vector.shape[1] < index.d:
        padding_size = index.d - query_vector.shape[1]
        query_vector = np.hstack([query_vector, np.zeros((query_vector.shape[0], padding_size), dtype='float32')])

    # --- 1. Get More Candidates ---
    k_candidates = 50  # Get 50 similar candidates to re-rank
    index.nprobe = 10
    distances, indices = index.search(query_vector, k_candidates)
    
    # --- 2. Re-Rank by Popularity ---
    candidate_indices = indices[0]
    # Filter out games the user already provided
    input_indices = {title_to_index[title.lower().strip()] for title in game_titles if title.lower().strip() in title_to_index}
    
    # Get the quality score for each candidate
    candidate_scores = df_games.iloc[candidate_indices][['name', 'quality_score']]
    
    # Filter out input games
    reranked_candidates = candidate_scores[~candidate_scores.index.isin(input_indices)]
    
    # Sort the candidates by their quality score
    final_recommendations = reranked_candidates.sort_values(by='quality_score', ascending=False)
    
    # --- 3. Select the Top 5 ---
    print(f"\n--- Recommendations based on your input ---")
    for i, (idx, row) in enumerate(final_recommendations.head(num_recs).iterrows()):
        print(f"{i+1}. {row['name']}")

# --- Now, test with your three games and ratings! ---
test_games = ["EA SPORTS FIFA 21", "EA SPORTS FC 24", "EA SPORTS FC 25"]
test_ratings = [10, 10, 10]

get_reranked_recommendations(test_games, test_ratings)

Query vector shape: (1, 50938)
Faiss index dimensionality: 1392


AssertionError: 

In [ ]:
#creating new vector

from sklearn.preprocessing import normalize
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# --- 1. Define Your New, Tuned Weights ---
keyword_weight = 1.5    # For tags, genres, etc.
publisher_weight = 2.0  # NEW: Very high weight to focus on publishers
semantic_weight = 2.0   # For description meaning
numerical_weight = 0.5  # For stats like quality and age

# --- 2. Create Keyword Soup (WITHOUT Publisher) ---
df_games['keyword_soup'] = df_games['tags_cleaned'] + ' ' + df_games['genres'] + ' ' + df_games['categories']

# --- 3. Process and Weigh Each Feature Set Separately ---
# Ensure 'keyword_soup' has no NaN values and convert to string
df_games['keyword_soup'] = df_games['keyword_soup'].fillna('').astype(str)

# Process and weigh KEYWORDS
tfidf_vectorizer_keywords = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 2))
keyword_vectors = tfidf_vectorizer_keywords.fit_transform(df_games['keyword_soup'])
weighted_keywords = keyword_vectors * keyword_weight

# Process and weigh PUBLISHERS
tfidf_vectorizer_publisher = TfidfVectorizer()
publisher_vectors = tfidf_vectorizer_publisher.fit_transform(df_games['publisher_cleaned'])
weighted_publishers = publisher_vectors * publisher_weight

# Process and weigh NUMERICALS
# Assuming 'numerical_vectors' is your pre-scaled NumPy array of numerical features
weighted_numericals = numerical_vectors * numerical_weight

# Weigh SEMANTICS
# Assuming 'embeddings_matrix' is your NumPy array of semantic vectors
weighted_semantics = embeddings_matrix * semantic_weight


# --- 4. Combine All Weighted Vectors ---
weighted_hybrid_vectors = hstack([
    weighted_keywords,
    weighted_publishers,  # Add the new, heavily weighted publisher vectors
    csr_matrix(weighted_numericals),
    csr_matrix(weighted_semantics)
])

# --- 5. Normalize and Finalize ---
final_vectors_normalized = normalize(weighted_hybrid_vectors, norm='l2', axis=1)
final_vectors = final_vectors_normalized.toarray().astype('float32')

print("Successfully created publisher-focused, weighted feature vectors.")
print("New shape:", final_vectors.shape)

NameError: name 'df_games' is not defined

In [ ]:
import numpy as np

# Assuming `final_vectors` is already created in your notebook
# Save the improved vectors to an .npy file
np.save("improved_version_2.npy", final_vectors)

# Verify the file was saved
print("Improved vectors saved to 'improved_version_2.npy'")

Improved vectors saved to 'improved_version_2.npy'


In [ ]:
##engine 3.1

import faiss
import numpy as np
import pandas as pd

# --- Load all necessary components ---
df_games = pd.read_csv("df_games.csv")  # Or your full cleaned dataset CSV
vectors = np.load("improved_version_2.npy")  # Load the improved hybrid vectors

# Verify the dimensions of the vectors
print(f"Shape of vectors: {vectors.shape}")

# Rebuild the Faiss index with the correct dimensions
d = vectors.shape[1]  # Number of dimensions
index = faiss.IndexFlatL2(d)  # L2 distance metric
index.add(vectors)  # Add the vectors to the index

title_to_index = pd.Series(df_games.index, index=df_games['name'])

def get_profile_recommendations(game_titles, ratings, k=6): # Added 'ratings' parameter
    """
    Finds and prints recommendations based on a weighted average profile of input games.
    """
    try:
        # Check if the number of games and ratings match
        if len(game_titles) != len(ratings):
            print("Error: The number of games and ratings must be the same.")
            return

        # 1. Check if all titles exist in the dataset
        missing_titles = [title for title in game_titles if title not in title_to_index.index]
        if missing_titles:
            print(f"Error: The following games were not found in the dataset: {', '.join(missing_titles)}")
            return
        
        # 2. Get the vectors for all input games
        input_vectors = [vectors[title_to_index[title]] for title in game_titles]
        
        # 3. Calculate the WEIGHTED average vector to create the "taste profile"
        query_vector = np.average(input_vectors, axis=0, weights=ratings).reshape(1, -1).astype('float32')
        
        # Verify the dimensions of the query vector
        print(f"Shape of query vector: {query_vector.shape}")

        # New line: Increase the search accuracy
        index.nprobe = 10 
        
        # 4. Search the Faiss index
        distances, indices = index.search(query_vector, k)
        
        print(f"--- Recommendations for a fan of {', '.join(game_titles)} ---")
        
        # 5. Print the results, filtering out the input games
        recs = []
        for i in range(k):
            rec_title = df_games.iloc[indices[0][i]]['name']
            if rec_title not in game_titles:
                recs.append(rec_title)
        
        for i, rec in enumerate(recs):
            print(f"{i+1}. {rec}")

    except KeyError as e:
        print(f"Error: Game {e} not found in the dataset.")

# --- Now, test with both games! ---
test_games = ["Red Dead Redemption 2", "Grand Theft Auto V", "Watch Dogs Legion"]
# Add a list of ratings (out of 10) to correspond to the games
test_ratings = [10, 9, 5]
# Call the function with the new ratings parameter
get_profile_recommendations(test_games, test_ratings)


Shape of vectors: (88886, 50938)
Shape of query vector: (1, 50938)
--- Recommendations for a fan of Red Dead Redemption 2, Grand Theft Auto V, Watch Dogs Legion ---
1. Red Dead Online
2. Red Dead Redemption
3. Grand Theft Auto III – The Definitive Edition
4. Grand Theft Auto Vice City – The Definitive Edition
